## Bias wave stability

Looking at the impact of RD changes on the bias wave\
Craig Lage - 05-Jan-23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
import lsst.afw.display as afwDisplay

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])
camera = butler.get('camera', instrument='LATISS')

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
bias = butler.get('bias', detector=0, exposure=2022110800013)
biasArr = bias.image.array
plt.title("AuxTel bias stability - Amp C10")
names = ['RD13-VBBon','RD13-VBBon','RD14.9-VBBoff','RD14.9-VBBon','RD14.9-VBBon','RD13-VBBon']
for i, expId in enumerate([2022110300020, 2023010500006, 2023010500018, 2023010500028, 2023010500078, 2023010500150]):
    exp = butler.get('raw', detector=0, exposure=expId)
    #biasExp = butler.get('bias', detector=0, exposure=expId)
    isrResult = isrTask.run(exp)
    arr = isrResult.exposure.image.array
    plt.plot(np.mean(arr[2100:3900, 0:400], axis=0), label=f"{expId}:{names[i]}")
    plt.ylim(-4,4)
plt.plot(np.mean(biasArr[2100:3900, 0:400], axis=0), ls='--', lw=2, color='black', label="Master bias")
plt.legend(bbox_to_anchor=(1.2, 0.5))
plt.xlabel("X coord (pixels)")
plt.ylabel("ADU")
plt.savefig("/home/c/cslage/u/AuxTel/isr/Bias_Stability_RD_Amp_C10_05Jan23.pdf")